# Grade: 100 points

# Assignment 03: Classification with Decision Trees, Ensemble Models and Neural Networks

## Instructions

Just like assignment 2, you will train various machine learning models for the binary classification task provided in the [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic) competition. Note: we will play loosely with the terms validation and test sets and may even use them interchangeably. This Kaggle competition has its own Test set file that you can make predications on and submit for scoring, so for us our "test" set could be viewed as a validation set.

The task is to predict whether an individual would survive the famous disaster. There are 11 input variables, and 2 output labels: **survived** or **did not survive**.

#### **Follow these steps before submitting your assignment:**

1. Complete the notebook.

2. Make sure all plots have axis labels.

3. Once the notebook is complete, `Restart` your kernel by clicking 'Kernel' > 'Restart & Run All'.

4. Fix any errors until your notebook runs without any problems.

5. Please note, a random seed of 42 needs to be set to ensure the reproducability of the results -- *DO NOT* change this random seed. **If you call additional functions that are based on random number generators, you will need to define their seed to 42 as well**.

6. Make sure to reference all external code and documentation used.

*Note: you may need to install XGBoost. You can do this by running the cell below.*

In [ ]:
#!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.decomposition import PCA

from xgboost import XGBClassifier

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
import torch.nn.utils as utils

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Q1 - Data Loading and Exploration

`https://www.kaggle.com/competitions/titanic` contains data on passengers of the famous Titanic disaster. The task is to examine and predict what sorts of people were more likely to survive using data on their name, age, gender, socio-economic class, etc.

#### 1. Loading our Data
Load the train data into a pandas DataFrames. Display the first few rows **of each DataFrame** and print the shape of the dataframe.

In [ ]:
#Q1.1

#### 2. Checking for Null Values

Check for null values in the training data.

In [ ]:
#Q1.2

#### 3. Checking the class balance
Since this is a binary classification task, class balance is extremely important. What percent of the training data belongs to each class? Is this roughly balanced?

In [ ]:
#Q1.3

You should see that the classes are roughly balanced, about a 60-40 split between not surviving and surviving.

# Q2 - Feature Engineering

1. Fill missing values for Age and Fare with their respective median values, make sure to round age.

In [ ]:
#Q2.1

2. Drop non-unique variables such as `PassengerId`, and irrelevant features such as `Ticket`, `Cabin`, and `Embarked`.

In [ ]:
#Q2.2

3. Combine `SibSp` and `Parch` into a single variable `Family`. Then drop `SibSp` and `Parch`.

In [ ]:
#Q2.3

Like Assignment 2, the `Name` column contains passengers' full names, including their titles (e.g., Mr., Mrs., Dr., Col.). Write code to **extract only the title** from each passenger's name. Standardize similar titles (e.g., Ms --> Miss, Mlle --> Miss, Mme --> Mrs), and group rare or honorific titles (e.g., Capt, Col, Dr, Sir, etc.) under a single label called "Special".

In [ ]:
special_names = ['Capt','Col','Countess','Don','Dona','Dr','Jonkheer','Lady','Major','Rev','Sir']
train['Name'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
train['Name'] = train['Name'].replace('Ms', 'Miss').replace('Mlle', 'Miss').replace('Mme', 'Mrs').replace(special_names, 'Special')

4. One-hot-encode the features `Name`, `Sex`, and `Pclass`.

In [ ]:
#Q2.4

5. Correlation Map
Visualize the correlation between variables using a heatmap. List the 3 features that are most related to a passenger's survival.

In [ ]:
#Q2.5 Correlation Map

In [ ]:
#Q2.5 List of three features

### Checkpoint 1

![Checkpoint 1](checkpoint_1.png)

Your `train.head()` should look like this.

In [ ]:
train.head()

# Q3 - Train/Test Split and Feature Scaling

#### 1. Scaling
Scale the train. Make sure you do not scale the one-hot-encoded data or the target variable (Survived).

In [ ]:
train.dtypes

In [ ]:
#Q3.1

Drop the target from the training data and add it to its own dataframe

In [ ]:
target = train['Survived']
train.drop(['Survived'], axis=1, inplace=True)

#### 2. Train/Test Split
Split the training data into a train and test set using an 80-20 split. Make sure you preserve the balance of the target variable using the `stratify = target` argument.

In [ ]:
#Q3.2

#### 3. Check the class balance of the train and test set.
Print the percent of samples that survived in both the train and test set. They should be almost equal since you stratified by the target variable when splitting the data.

In [ ]:
#Q3.3

# Q4 - Decision Tree Training

Decision trees have a number of parameters to adjust. For this assignment we have chosen `criterion`, `max_depth`, `max_features`, and `splitter` as the parameters to adjust. You will exhaustively search over some values of these parameters using a grid search.

The grid and potential parameter values are defined below. For more information on what these parameters do, see the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [ ]:
# Hyper parameters for grid search
param_grid = {'criterion' : ['gini', 'entropy'] # The function to measure the quality of a split.
              , 'max_depth' : [1, 2, 3, 4, 5, None] # None means the tree can grow arbitrarily deep.
              , 'max_features' : [2, 3, 4, 'sqrt', 'log2', None] # The number of features to consider when looking for the best split.
              , 'splitter' : ['best', 'random'] # The strategy used to choose the split at each node.
             }

#### 1. Instantiate a `DecisionTreeClassifier` model.

In [ ]:
#Q4.1

#### 2. Conduct the grid search.

Fit the grid search object on the training data and print the best cross-validation score and best parameters. Use the same parameters for the grid search as in lab 6 for results consistent with mine.

In [ ]:
#Q4.2

#### 3. Evaluate on the validation set.
Using the best parameters, evaluate the model on the validation set. Calculate the accuracy and AUC on the validation set.

In [ ]:
#Q4.3

### Checkpoint 2

Your accuracy for this decision tree should be between ~70-83%. It it not controlled via a random seed so results will vary!

# Q5 - Ensemble Models

As we saw in lecture, a single decision tree can be unstable. We can use ensemble models to get more accurate predictions. You will train an XGBoost model which combines multiple trees to see if you can get a better performance. As with decision trees, there are a number of tunable parameters, so you will use a grid search to find the "best" ones.

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],        # number of boosting rounds
    'max_depth': [3, 4, 5, 6],              # depth of each tree
    'learning_rate': [0.01, 0.05, 0.1],     # step size shrinkage
    'subsample': [0.8, 1.0],                # fraction of samples used per tree
    'colsample_bytree': [0.8, 1.0],         # fraction of features used per tree
    'gamma': [0, 0.5, 1],                   # minimum loss reduction to make a split
    'reg_lambda': [1, 5, 10]                # L2 regularization strength
}

#### 1. Instantiate an XGBoost model.

Use the same parameters for the grid search as in lab 6 for results consistent with mine.

In [ ]:
xgb = XGBClassifier(random_state = 42)
#Q5.1

#### 2. Conduct the grid search.

Fit the grid search object on the training data and print the best cross-validation score and best parameters.

In [ ]:
#Q5.2

#### 3. Evaluate on the validation set.
Using the best parameters, evaluate the model on the validation (test) set. Calculate the accuracy and AUC score on the validation set.

In [ ]:
#Q5.3

### Checkpoint 3

Your XGboost model should be 83.24% accuracy with and AUC of 0.81. This is controlled by a random seed. If you do not get this value but are close, that is fine too.

# Q6 - Training a Neural Network

The last model you will train is a Neural Network. The Neural Network (NN) architecture will consist of 4 layers:
1. Input (size = 8)
2. Hidden1 (size = 16)
3. Hidden2 (size = 16)
4. Output layer (size = 2)

This is a simple network that uses Relu activation function on each of the hidden layers, and a Sigmoid function on the output.

You will train your model using Binary Cross Entropy Loss for 50 epochs, with an Adam optimizer.

#### 1. Define the NN architecture.

Complete the code below to define the NN architecture. You should have 4 layers, with activation functions between each layer.
1. Input layer (linear)
2. Apply ReLU
3. First hidden layer (linear)
4. Apply Relu
5. Second hidden layer (linear)
6. Apply Sigmoid

Do not edit the `forward` function. The code you write should work without having to edit it.

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        #Q6.1

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

#### 2. Prepare the data for training.

Create tensor datasets from `X_train` and `y_train` and from `X_test` and `y_test`. To do this, first convert them to Pytorch tensors. Then turn these into DataLoaders with a batch size of 64. Make sure you shuffle the train loader but do not shuffle the test loader.

In [ ]:
# Convert your data to tensors
#Q6.2 Create tensor datasets

In [ ]:
#Q6.2 Define Dataloaders, you can combine these two steps into a single cell if you'd like.

#### 4. Prepare for training

1. Before instantiating your model, you will need to specify the size of each layer.
2. Instantiate your model.
3. Define your criterion, this if your loss function (remember we are doing binary classification).
4. Declare your optimizer, use `optim.Adam` for this with a learning rate of `0.001`.

In [ ]:
#Q6.4 1-4.

#### 5. Train your NN

Use the training loop below to train your neural network. The loop keeps track of the training loss and accuracy so we can track their progress and plot them in the next question. There is no coding for you to do here!

In [ ]:
num_epochs = 50
train_losses, val_losses = [], []
train_accuracies, test_accuracies = [], []

# Training loop
for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad() # Zero out the gradients
        outputs = model(xb).squeeze(1) # Get the predictions
        loss = criterion(outputs, yb) # Calculate the loss
        loss.backward() # Backpropagation
        optimizer.step() # Optimize
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation loop
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for xb, yb in test_loader:
            outputs = model(xb).squeeze(1)
            loss = criterion(outputs, yb)
            val_loss += loss.item()
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(yb.cpu().numpy())
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    auc = roc_auc_score(all_labels, all_preds)
    preds_binary = (torch.tensor(all_preds) > 0.5).float()
    acc = accuracy_score(all_labels, preds_binary)
    if epoch % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | AUC: {auc:.4f} | Acc: {acc:.4f}")

#### 6. Plot the training loss and accuracy

Create a plot for the train and test (validation) loss over the epochs.

In [ ]:
#Q6.6

#### 7. Final Evaluation on the Test Set

Use the code below to calculate the test accuracy and AUC score. Again, no code for you to add here!

In [ ]:
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

all_preds = []   # for AUC (raw sigmoid outputs)
all_labels = []  # true labels

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        outputs = model(X_batch).squeeze()  # sigmoid probabilities
        loss = criterion(outputs, y_batch)

        test_loss += loss.item() * X_batch.size(0)

        # Store for AUC
        all_preds.extend(outputs.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

        # Accuracy (thresholded at 0.5)
        preds = (outputs >= 0.5).float()
        test_correct += (preds == y_batch).sum().item()
        test_total += y_batch.size(0)

# Compute averages
avg_test_loss = test_loss / test_total
acc_nn = test_correct / test_total

# Compute AUC
auc_nn = roc_auc_score(all_labels, all_preds)

print(f"\nTest Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {acc_nn:.4f}")
print(f"Test AUC: {auc_nn:.4f}")

# Q7 - Model Evaluation & Comparison

#### 1. Accuracy and AUC Comparison
Display the accuracy and AUC of each model in a table using the `display()` function.

In [ ]:
#Q7.1

#### Discussion

State which model performed best. If you wanted to attempt to improve performance of any of these models' performance what would you try and what is the reasoning behind your decision?

Discussion answer here.